In [9]:
import torch
import pandas as pd
import numpy as np

In [10]:
df = pd.read_csv('shortURL/dataset_phishing.csv')

In [11]:
df['phishing'] = (df['status'] == 'phishing')

df.drop('status', inplace=True, axis=1)

In [13]:
df_URL = df[['url', 'length_url', 'phishing']]

In [14]:
df_URL[:10]

,url,length_url,phishing
0,http://www.crestonwood.com/router.php,37,False
1,http://shadetreetechnology.com/V4/validation/a...,77,True
2,https://support-appleld.com.secureupdate.duila...,126,True
3,http://rgipt.ac.in,18,False
4,http://www.iracing.com/tracks/gateway-motorspo...,55,False
5,http://appleid.apple.com-app.es/,32,True
6,http://www.mutuo.it,19,False
7,http://www.shadetreetechnology.com/V4/validati...,81,True
8,http://vamoaestudiarmedicina.blogspot.com/,42,False
9,https://parade.com/425836/joshwigler/the-amazi...,104,False


In [21]:
#토크나이징을 위한 url 분할
str_url=df_URL['url'][0]

list_url=list(str_url)

In [20]:
len(list_url)

37

In [19]:
list_url

['h',
 't',
 't',
 'p',
 ':',
 '/',
 '/',
 'w',
 'w',
 'w',
 '.',
 'c',
 'r',
 'e',
 's',
 't',
 'o',
 'n',
 'w',
 'o',
 'o',
 'd',
 '.',
 'c',
 'o',
 'm',
 '/',
 'r',
 'o',
 'u',
 't',
 'e',
 'r',
 '.',
 'p',
 'h',
 'p']